In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow_gnn as tfgnn
import numpy as np
import tensorflow as tf

In [ ]:
graphs = []

for i in range(100):
    g = nx.random_tree(np.random.poisson(10))

    graphs.append((1, g))
for i in range(100):
    g = nx.erdos_renyi_graph(np.random.poisson(10),0.2)
    graphs.append((0,g))

In [ ]:
def nx_graph_to_tfgnn(graph, label) -> tfgnn.GraphTensor : 
    n_nodes = len(graph.nodes)
    n_edges = len(graph.edges)

    tf_g = tfgnn.GraphTensor.from_pieces(
    node_sets={
        "normal": tfgnn.NodeSet.from_fields(
            sizes=tf.constant([n_nodes]),
            features={
                "size":tf.random.normal((n_nodes,3))
                # "size": tf.ones((n_nodes, 3))
                # tfgnn.HIDDEN_STATE:tf.TensorSpec((None,7), tf.Float32)
            }
        )
    },

    edge_sets={
        "connects": tfgnn.EdgeSet.from_fields(
            sizes=tf.constant([n_edges]),
            adjacency=tfgnn.Adjacency.from_indices(
                target=("normal",[i[1] for i in list(graph.edges)]),
                source=("normal",[i[0] for i in list(graph.edges)]),
            )
        )
    },
    context=tfgnn.Context.from_fields(
        features={"label":[label]}
    )
    )
    
    return tf_g

In [ ]:
g = nx_graph_to_tfgnn(graphs[0][1], graphs[0][0])

In [ ]:
g = g.merge_batch_to_components()